In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline

In [22]:
# veri kaynağı: http://veribilimi.co/data/iris.csv
df = pd.read_csv("http://veribilimi.co/data/iris.csv")

# 1. Veri keşfi

In [23]:
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [25]:
# Boş değer yok

In [26]:
df.describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [27]:
# Aykırı değer görünmüyor.

In [28]:
df.groupby(['Species']).agg(['mean']).head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
,mean,mean,mean,mean
Species,,,,
Iris-setosa,5.006,3.418,1.464,0.244
Iris-versicolor,5.936,2.770,4.260,1.326
Iris-virginica,6.588,2.974,5.552,2.026


# 2. Veri Temizliği ve Ön Hazırlığı

In [29]:
# Veri önceden temizlenmiş. Ayrıca kategorik değişken de yok.
# Bu aşamada birşey yapmıyoruz.

## 2.1. Veriyi X,y olarak bölme (nitelikler matrisi ve hedef değişken - dikey bölme)

In [30]:
# hedef nitelik en sonda. -1' kadar hepsini seçiyoruz (:-1).
X = df.iloc[:,:-1].values
X.shape

(150, 4)

In [31]:
# Kontrol
X[:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [32]:
# Hedef nitelik (Species) en sonda (-1)
y = df.iloc[:,-1].values
y.shape

(150,)

In [33]:
# Kontrol
y[:10]

array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa'], dtype=object)

## 2.2. Kategorik Nitelik Dönüştürme

In [34]:
# Kategorik nitelik olarak sadece hedef niteik var (y)

### 2.2.1. Hedef değişken için Label Encoder

In [35]:
# Hedef niteliği sadece LabelEncoder ile rakamlara kodluyoruz.
from sklearn.preprocessing import LabelEncoder
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

In [36]:
# Kontrol
y[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [37]:
# OneHotEncoder yapmıyoruz. Çünkü hedef değişken.

## 2.3. Feature Scaling (Standardizasyon)

In [38]:
# Yapmak zorunda değiliz çünkü girdilerin hepsi aynı ölçekte

## 2.4. Veriyi eğitim ve test setlerine bölme (yatay bölme)

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=142)

In [41]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(112, 4)
(38, 4)
(112,)
(38,)


# 3. Model Oluşturma ve Eğitim

In [42]:
from sklearn.neighbors import KNeighborsClassifier

In [43]:
classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
# minkowski: mesafe metriği
# n_neighbors: kaç komşuya bakılıp kararverilecek, tek sayı olmalı
# p: eğer minkowski seçersek hangi yöntemi olsun

In [44]:
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

# 4. Predictions

In [45]:
y_pred = classifier.predict(X_test)

## 4.1. Tahmin sonuçlarını numpy.ndarray'den dataframe yapma (İncelemek daha Kolay)

In [57]:
predictions = pd.DataFrame(data=y_pred,    # values
                index=range(len(y_pred)),    # 1st column as index
                   columns=['y_pred'])  # 1st row as the column names

# Sadece y_pred'den oluşan df'e test(gerçek) y_test'i sütun olarak ekleme
predictions['y_test'] = y_test
predictions.head()

,y_pred,y_test
0,0,0
1,1,1
2,1,1
3,2,2
4,1,1


# 5. Model Değerlendirme

## 5.1. Hata Matrisi (Confusion Matrix)

In [58]:
from sklearn.metrics import confusion_matrix

In [59]:
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(cm)

[[ 7  0  0]
 [ 0 15  2]
 [ 0  2 12]]


In [60]:
from sklearn.metrics import accuracy_score

In [61]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: % {:10.2f}".format(accuracy*100)) 
# :10.2f ifadesi ondalık kısımda iki basamak göstermek için

Accuracy: %      89.47


## 5.2. Elle Accuracy hesaplama

In [62]:
predictions[predictions['y_pred'] != predictions['y_test']]

,y_pred,y_test
14,2,1
18,1,2
25,1,2
34,2,1


In [63]:
predictions[predictions['y_pred'] != predictions['y_test']].count()

y_pred    4
y_test    4
dtype: int64

In [64]:
len(y_test)

38

In [65]:
accuracy = (len(y_test) - 4) / len(y_test)
accuracy

0.8947368421052632

## 5.3. En iyi k değeri

In [67]:
# Üretilecek accuracy değerlerini toplayacağımız liste
accuracy_list = []

# k değerlerinin toplanacağı liste
k_list = []

# Arzu edilen k değerleri boyunca döngü çalıştır ve her bir k değeri için
# elde edilen accuracy skorlarını ve k değerlerini birer listede topla
for k in range(2,10):
    #sınıflandırıcı küme sayısı olarak k alıyor
    classifier = KNeighborsClassifier(n_neighbors=k, metric='minkowski', p=2)
    # Eğitim
    classifier.fit(X_train, y_train)
    # Tahmin
    y_pred = classifier.predict(X_test)
    # Accuracy skoru
    accuracy = accuracy_score(y_test, y_pred)
    # Bu döngüde elde edilen accuracy skorunu listeye ekle
    accuracy_list.append(accuracy)
    # Bu döngüde kullanılan k değerini listeye ekle
    k_list.append(k)
    # yapılan işi görmek için döngüde kullanılan k, accuracy çiftini ekrana yaz
    print("{} için accuracy: {:10.2f}".format(k,accuracy))
    
    
    
    

2 için accuracy:       0.89
3 için accuracy:       0.92
4 için accuracy:       0.92
5 için accuracy:       0.89
6 için accuracy:       0.95
7 için accuracy:       0.95
8 için accuracy:       0.95
9 için accuracy:       0.95


In [69]:
# Accuracy skorlarının bulunduğu listeden en yüksek skorun indeks numarasını al 
acc_max_index = accuracy_list.index(max(accuracy_list))
acc_max_index

4

In [70]:
# Yukarıdaki indeks k listesinde hangi k sayısına (değere) karşılık geliyorsa
# bu k değeri en iyi sonucu (accuracy) üretiyor.
k_list[acc_max_index]

6

## 5.4. En iyi k değeri için modeli eğitelim

In [71]:
# find index of max value inaccuracy_list
acc_max_index = accuracy_list.index(max(accuracy_list))
# assing this index to k_list
k = k_list[acc_max_index]
classifier = KNeighborsClassifier(n_neighbors=k, metric='minkowski', p=2)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=6, p=2,
           weights='uniform')

## 5.5. En iyi modelle tahmin yapma

In [72]:
# test setinden tahminler yapalım
y_pred = classifier.predict(X_test)

In [73]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: % {:10.2f}".format(accuracy*100)) 
# :10.2f ifadesi ondalık kısımda iki basamak göstermek için

Accuracy: %      94.74


## 5.6. Elle tahmin seti üretme

In [74]:
# test seti dışında elle tahmin seti oluşturalım
X_manual_test = np.array([4.0,4.0,4.0,4.0]).reshape(1,4)

# array shape mutlaka uygun olmalı
print(X_manual_test.shape)
print(X_test.shape)

(1, 4)
(38, 4)


In [75]:
classifier.predict(X_manual_test)

array([1])

In [76]:
# Bu 1 hangi sınıtı?
labelencoder_y.inverse_transform(classifier.predict(X_manual_test))

array(['Iris-versicolor'], dtype=object)

# 6. Modeli Saklama ve Tekrar Kullanma

In [77]:
# Saklamadan önce tüm veri seti ile modeli tekrar eğitelim

## 6.1. Tüm veri ile modeli son olarak eğitme

In [79]:
# find index of max value inaccuracy_list
acc_max_index = accuracy_list.index(max(accuracy_list))
# assing this index to k_list
k = k_list[acc_max_index]

classifier_last = KNeighborsClassifier(n_neighbors=k, metric='minkowski', p=2)
# Eğitim setlerinin X ve y olduğuna dikkat!!!
classifier_last.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=6, p=2,
           weights='uniform')

## 6.2. Modeli kaydetme

In [80]:
from sklearn.externals import joblib
# joblib.dump(sınıflandırıcı, "/dizin/saklamak_istedigimiz_isim.pkl")
joblib.dump(classifier_last, "01.knn_with_iris_dataset.pkl")

['01.knn_with_iris_dataset.pkl']

In [81]:
# Kaydedilen yere Dosya gezgini veya jupyter home'dan bir göz atalım. 

## 6.3. Modeli okuma

In [82]:
classifier_last_loaded = joblib.load("01.knn_with_iris_dataset.pkl")

In [83]:
# Kontrol
classifier_last_loaded.classes_

array([0, 1, 2])